# ResNet Training with top-50 artists

In [0]:
# model and learning
import numpy as np
import h5py
import cv2 as cv


%tensorflow_version 2.x
import tensorflow as tf
import keras as ks

from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.optimizers import SGD
from keras.callbacks import CSVLogger


import os as os
import pathlib as pl
from PIL import ImageFile

from google.colab import drive

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.2.0


Using TensorFlow backend.


In [0]:
drive.mount('/content/drive')
my_data_dir = '/content/drive/My Drive/DataForColab/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## GPU Set Up

In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Model

In [0]:
def main(withGPU=True):
    TRAIN_LOCATION = my_data_dir + 'ArtworksReducedTVT/train'
    VAL_LOCATION = my_data_dir + 'ArtworksReducedTVT/val'
    TRAIN_IMAGES = pl.Path(TRAIN_LOCATION)
    VAL_IMAGES = pl.Path(VAL_LOCATION)
    CLASSES = [item.name for item in TRAIN_IMAGES.glob('*')]
    CLASSES.sort()
    BATCH_SIZE = 256
    EPOCHS = 20
    IMG_WIDTH = 200
    IMG_HEIGHT = 200
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    image_generator = ks.preprocessing.image.ImageDataGenerator(rescale=1. / 255,
                                                                featurewise_center=True,
                                                                featurewise_std_normalization=True)

    image_generator.mean = np.array([0.485, 0.456, 0.406], dtype=np.float32).reshape((1, 1, 3))  # ordering: [R, G, B]
    image_generator.std = np.array([0.229, 0.224, 0.225], dtype=np.float32).reshape((1, 1, 3))

    train_data_gen = image_generator.flow_from_directory(directory=TRAIN_IMAGES,
                                                         batch_size=BATCH_SIZE,
                                                         shuffle=True,
                                                         target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                         color_mode='rgb',
                                                         class_mode='categorical',
                                                         classes=CLASSES,
                                                         interpolation='lanczos')
    
    val_data_gen = image_generator.flow_from_directory(directory=VAL_IMAGES,
                                                       batch_size=BATCH_SIZE,
                                                       shuffle=True,
                                                       target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                       color_mode='rgb',
                                                       class_mode='categorical',
                                                       classes=CLASSES,
                                                       interpolation='lanczos')
    

    input_tensor = ks.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    base_model = InceptionResNetV2(input_tensor=input_tensor, weights='imagenet', include_top=False)

    # add a global spatial average pooling layer
    x = base_model.output
    x = ks.layers.GlobalAveragePooling2D(data_format="channels_last")(x)
    x = ks.layers.BatchNormalization()(x)
    # TODO: ADD FLATTEN
    # let's add a fully-connected layer
    x = ks.layers.Dense(256, activation='relu')(x)
    x = ks.layers.Dropout(rate=0.2)(x)
    x = ks.layers.Dense(256, activation='relu')(x)
    # and a logistic layer
    predictions = ks.layers.Dense(len(CLASSES), activation='softmax')(x)

    csv_logger = CSVLogger(my_data_dir + "model_history_log.csv", append=True)

    if withGPU:
      with tf.device('/device:GPU:0'):
        model = ks.Model(inputs=base_model.input, outputs=predictions)
        
        for layer in base_model.layers:
          layer.trainable = False
          
        model.compile(loss='categorical_crossentropy',
                      optimizer=ks.optimizers.Adam(lr=1e-04),
                      metrics=['accuracy', 'top_k_categorical_accuracy'])
    else:
      model = ks.models.Model(inputs=base_model.input, outputs=predictions)
      for layer in base_model.layers:
        layer.trainable = False
      model.compile(loss='categorical_crossentropy',
                    optimizer=ks.optimizers.Adam(lr=1e-04),
                    metrics=['accuracy', 'top_k_categorical_accuracy'])

    model.fit_generator(train_data_gen, epochs=EPOCHS, validation_data=val_data_gen,
                      callbacks=[csv_logger], verbose=1)

    model.save(my_data_dir + "save_model_top50_resnet_longerTraining")

    return model, train_data_gen

In [0]:
model_output, data_gen = main(True)

Found 27806 images belonging to 50 classes.
Found 3344 images belonging to 50 classes.
Epoch 1/20
109/109 [==============================] - 324s 3s/step - loss: 3.7449 - accuracy: 0.0943 - top_k_categorical_accuracy: 0.2678 - val_loss: 3.7349 - val_accuracy: 0.1181 - val_top_k_categorical_accuracy: 0.3047
Epoch 2/20
109/109 [==============================] - 304s 3s/step - loss: 3.1805 - accuracy: 0.2100 - top_k_categorical_accuracy: 0.4686 - val_loss: 3.6225 - val_accuracy: 0.1800 - val_top_k_categorical_accuracy: 0.4208
Epoch 3/20
109/109 [==============================] - 310s 3s/step - loss: 2.8368 - accuracy: 0.2821 - top_k_categorical_accuracy: 0.5702 - val_loss: 4.3075 - val_accuracy: 0.2285 - val_top_k_categorical_accuracy: 0.5024
Epoch 4/20
109/109 [==============================] - 311s 3s/step - loss: 2.5986 - accuracy: 0.3326 - top_k_categorical_accuracy: 0.6290 - val_loss: 3.4631 - val_accuracy: 0.2545 - val_top_k_categorical_accuracy: 0.5475
Epoch 5/20
109/109 [=========